In [1]:
# Install blosc2 and caterva2 in Pyodide environments
import sys
if sys.platform == "emscripten":
    # Install latest blosc2
    import requests
    import micropip
    url = "https://api.github.com/repos/Blosc/python-blosc2/releases/latest"
    resp = requests.get(url, headers={"Accept": "application/vnd.github+json"})
    latest_tag = resp.json()["tag_name"].strip('v')
    wheel = f'https://cat2.cloud/demo/media/blosc2-{latest_tag}-cp312-cp312-pyodide_2024_0_wasm32.whl'
    await micropip.install(wheel)
    %pip install caterva2

In [2]:
import blosc2
import caterva2 as cat2

### Caterva2: remote and seamless data management and computation

In [3]:
%%time
# c = cat2.Client("https://cat2.cloud/demo")
c = cat2.Client("https://cat2.cloud/demo", ("user@example.com", "foobar11"))
myroot = c.get("@public")
dset = myroot["large/gaia-3d.b2nd"]
print(type(dset), dset.shape, dset.dtype)

<class 'caterva2.client.Dataset'> (20000, 20000, 20000) uint8
CPU times: user 24.2 ms, sys: 5.8 ms, total: 30 ms
Wall time: 248 ms


In [4]:
%%time
# Get stars in 100 light-years cube around our Sun
dslice = dset[9_950:10_050, 9_950:10_050, 9_950:10_050]
print("Retrieved slice shape", dslice.shape)
print("Total stars near us (computed locally):", dslice.sum())

Retrieved slice shape (100, 100, 100)
Total stars near us (computed locally) 996
CPU times: user 2.16 ms, sys: 963 μs, total: 3.13 ms
Wall time: 53.6 ms


In [5]:
%%time
# Same, but doing the computation in the server (only works when authenticated)
lexpr = blosc2.lazyexpr('dset[9_950:10_050, 9_950:10_050, 9_950:10_050].sum()')
sum_path = c.upload_lazyexpr('@personal/sum_dslice.b2nd', expression=lexpr)
dsum = c.fetch(sum_path)
print("Total stars near us (computed remotely):", dsum[()])

Total stars near us (computed remotely) 996
CPU times: user 5.96 ms, sys: 4.71 ms, total: 10.7 ms
Wall time: 182 ms


/Users/faltet/miniforge3/envs/blosc2/lib/python3.13/ast.py:422: UserWarning: Function shape parser not implemented for slice.
  return visitor(node)
